<a href="https://colab.research.google.com/github/davidnalbandyann/ACA_homework/blob/main/Decision_Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementation

In [ ]:
class DecisionTree:
    def __init__(self):
        pass

    def fit(self, X, y):
        tree_dict = dict()
        best_gini = 1.1
        for f in range(X.shape[1]):
            values = np.unique(X[:, f])
            for v in values[1:]:
                y1 = y[X[:, f]>=v]
                y2 = y[X[:, f]<v]
                gini = self.gini(np.unique(y1, return_counts=True)[1], np.unique(y2, return_counts=True)[1])
                if gini<best_gini:
                    tree_dict['column'] = f
                    tree_dict['value'] = v
                    best_gini = gini

        if 'column' not in tree_dict:
            return stats.mode(y)[0][0]

        f = tree_dict['column']
        v = tree_dict['value']
        y1 = y[X[:, f]>=v]
        y2 = y[X[:, f]<v]

        if len(np.unique(y1))==1:
            tree_dict['true'] = y1[0]
        else:
            tree_dict['true'] = self.fit(X[X[:, f]>=v], y1)

        if len(np.unique(y2))==1:
            tree_dict['false'] = y2[0]
        else:
            tree_dict['false'] = self.fit(X[X[:, f]<v], y2)

        self.tree_dict = tree_dict
        return tree_dict

    def get_value(self, x, tree):
        if not isinstance(tree, dict):
            return tree
        if x[tree['column']] >= tree['value']:
            return self.get_value(x, tree['true'])
        return self.get_value(x, tree['false'])

    def predict(self, X):
        y_pred = []
        for x in X:
            y_pred.append(self.get_value(x, self.tree_dict))
        return y_pred

    def gini(self, y1, y2):
        gini1 = 1 - ((y1/y1.sum()) ** 2).sum()
        gini2 = 1 - ((y2/y2.sum()) ** 2).sum()
        return (gini1 * y1.sum() + gini2 * y2.sum()) / (y1.sum()+y2.sum())